In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from lxml import html
from urllib.request import urlopen
import regex as re
import json
from pandas.io.json import json_normalize
import glob

In [2]:
list_of_websites = glob.glob('./data/webs/*')

#see how many restaurant do we have:
len(list_of_websites)

1047

In [3]:
def restaurant_data():
    for web in list_of_websites:
        try:
            url = pd.read_html(web)
            url = url[0][1][30]
            restaurant = str(re.search(r'<script id="restaurant_jsonld"(.*)}</script><div class="', url))
            b = re.sub("<regex.Match object; span=(.*), match=\'<script id=", '', restaurant)
            c = b.replace('"restaurant_jsonld" type="application/ld+json">', '')
            d = c.replace('</script><div class="', '')
            e = d.replace("'>",'')
            data = json.loads(e)
            json_file = json_normalize(data)
            restaurant_name = str(json_file['name'])


            rest = restaurant_name.replace('\nName: name, dtype: object','')
            restaurant_cor = rest.replace('0    ', '')
            restaurant_correct = restaurant_cor.replace('Menú del restaurante ', '')
            plates = []
            num=[]

            search_rating = str(re.search(r'Ratings":{"thefork":{"ratingValue":[+-]?([0-9]*[.])?[0-9]+,"reviewCount":[+-]?([0-9]*[.])?[0-9]+}', url))
            almost_rating = re.sub("<regex.Match object; span=(.*), match=\'Ratings", '', search_rating)
            rate_b = almost_rating.replace('":{"thefork":{"ratingValue":', '')
            rate_c = rate_b.replace('"reviewCount":', '')
            rate_d = rate_c.replace("'>", '')
            rate_e = rate_d.replace('}', '')
            rate_f = re.sub(',[+-]?([0-9]*[.])?[0-9]+', '', rate_e)

            search_cuisine = str(re.search(r'"servesCuisine":"\w*"', url))
            almost_cuisine = re.sub("<regex.Match object; span=(.*), match=\'", '', search_cuisine)
            cuisine_b = almost_cuisine.replace('"servesCuisine":', '')
            cuisine= cuisine_b.replace("'>", '')

            search_street = str(re.search(r'"address":{"street":"\w+( +\w+)*', url))
            almost_street = re.sub("<regex.Match object; span=(.*), match=\'", '', search_street)
            street_b = almost_street.replace('address":{"street":"', '')
            street = street_b.replace("'>", '')

            search_geo = str(re.search(r'<script>window.__INITIAL_STATE__ = (.*);</script>', url))
            geo_b = re.sub("<regex.Match object; span=(.*), match='<script>window.__INITIAL_STATE__ = ", '', search_geo)
            geo_c = geo_b.replace("</script><script>window.__APOLLO_STATE__={};</script>'>", '')
            almost_latitude = re.findall(r'"geo":{"latitude":[-+]?\d*\.*\d+,"longitude":[-+]?\d*\.*\d+', geo_c)

            col_plate=[]
            for i in range(0, len(data['hasMenuSection'])):
                col_plate.append(str(json_file['hasMenuSection'][0][i]['name']))
                plates.append(json_file['hasMenuSection'].to_dict()[0][i]['hasMenuItem'])
                num.append(str(i)+str(restaurant_name))

            for j in range(0,len(plates)):
                df1 = pd.json_normalize(plates[j])
                df1['type_of_plate'] = col_plate[j]
                df1['rating'] = rate_f
                df1['cuisine'] = cuisine
                df1['direction'] = street
                df1['geo'] = almost_latitude[0]
                df1.to_csv('./data/csv_each_restaurants/'+str(j)+str(restaurant_correct)+'.csv')


            for file in glob.glob('./data/csv_each_restaurants/*.csv'):
                if restaurant_correct in file:
                    df = pd.read_csv(file)
                    df['restaurant'] = restaurant_correct
                    df.to_csv(file)
        except:
            pass

In [4]:
restaurant_data()

<ipython-input-3-b99969d5fdb0>:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  json_file = json_normalize(data)


In [6]:
dfs = glob.glob('./data/csv_each_restaurants/*.csv')

result = pd.concat([pd.read_csv(df) for df in dfs], ignore_index=True)

result.to_csv('merge1.csv')

In [7]:
df_rest = pd.read_csv('merge1.csv')
df_rest

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,@type,name,offers.@type,offers.price,offers.priceCurrency,type_of_plate,rating,cuisine,direction,geo,restaurant,description,Unnamed: 0.1.1.1
0,0,0,0,MenuItem,Misto Tagliatella,Offer,15.30,EUR,Gratinati e Risotto,8.7,"""Italiano""","""Calle de Fuencarral","""geo"":{""latitude"":40.4310901,""longitude"":-3.70347",La Tagliatella - Fuencarral,NaN,NaN
1,1,1,1,MenuItem,Cannelloni,Offer,13.85,EUR,Gratinati e Risotto,8.7,"""Italiano""","""Calle de Fuencarral","""geo"":{""latitude"":40.4310901,""longitude"":-3.70347",La Tagliatella - Fuencarral,NaN,NaN
2,2,0,0,MenuItem,Botella de champán,NaN,NaN,NaN,Bebidas,9,"""Americano""","""C","""geo"":{""latitude"":40.418802280939,""longitude"":...",La Gamella,NaN,NaN
3,3,1,1,MenuItem,Copa de champán,Offer,11.00,EUR,Bebidas,9,"""Americano""","""C","""geo"":{""latitude"":40.418802280939,""longitude"":...",La Gamella,NaN,NaN
4,4,2,2,MenuItem,Café,Offer,2.10,EUR,Bebidas,9,"""Americano""","""C","""geo"":{""latitude"":40.418802280939,""longitude"":...",La Gamella,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27173,27173,2,2,MenuItem,California crab,NaN,NaN,NaN,Uramaki - 8 piezas,None,"""Japonés""","""Plaza Daoiz y Velarde","""geo"":{""latitude"":40.4020666,""longitude"":-3.67...",Sushi and Beer,NaN,NaN
27174,27174,3,3,MenuItem,Eni fry,NaN,NaN,NaN,Uramaki - 8 piezas,None,"""Japonés""","""Plaza Daoiz y Velarde","""geo"":{""latitude"":40.4020666,""longitude"":-3.67...",Sushi and Beer,NaN,NaN
27175,27175,0,0,MenuItem,Fabes de Tineo con su compango,Offer,17.82,EUR,Plato,9.1,"""Asturiano""","""C","""geo"":{""latitude"":40.457086513128,""longitude"":...",Teitu,NaN,NaN
27176,27176,1,1,MenuItem,Rape estilo Lastres,Offer,17.82,EUR,Plato,9.1,"""Asturiano""","""C","""geo"":{""latitude"":40.457086513128,""longitude"":...",Teitu,NaN,NaN


In [8]:
df_rest = df_rest.loc[:, ~df_rest.columns.str.contains('^Unnamed')]
df_rest = df_rest.drop('description', axis=1)

In [9]:
df_rest = df_rest.drop('@type', axis=1)
df_rest = df_rest.drop('offers.@type', axis=1)
df_rest = df_rest.drop('offers.priceCurrency', axis=1)

In [10]:
df_rest.dropna(inplace=True)

In [11]:
df_rest

,name,offers.price,type_of_plate,rating,cuisine,direction,geo,restaurant
0,Misto Tagliatella,15.30,Gratinati e Risotto,8.7,"""Italiano""","""Calle de Fuencarral","""geo"":{""latitude"":40.4310901,""longitude"":-3.70347",La Tagliatella - Fuencarral
1,Cannelloni,13.85,Gratinati e Risotto,8.7,"""Italiano""","""Calle de Fuencarral","""geo"":{""latitude"":40.4310901,""longitude"":-3.70347",La Tagliatella - Fuencarral
3,Copa de champán,11.00,Bebidas,9,"""Americano""","""C","""geo"":{""latitude"":40.418802280939,""longitude"":...",La Gamella
4,Café,2.10,Bebidas,9,"""Americano""","""C","""geo"":{""latitude"":40.418802280939,""longitude"":...",La Gamella
6,Media botella de agua,3.00,Bebidas,9,"""Americano""","""C","""geo"":{""latitude"":40.418802280939,""longitude"":...",La Gamella
...,...,...,...,...,...,...,...,...
27169,Wagyu tataki,85.00,Wagyu,9.2,"""Japonés""","""Calle Marqués de Villamagna","""geo"":{""latitude"":40.4298681,""longitude"":-3.68...",Sushi Bar Hannah
27170,Wagyu hoba,85.00,Wagyu,9.2,"""Japonés""","""Calle Marqués de Villamagna","""geo"":{""latitude"":40.4298681,""longitude"":-3.68...",Sushi Bar Hannah
27175,Fabes de Tineo con su compango,17.82,Plato,9.1,"""Asturiano""","""C","""geo"":{""latitude"":40.457086513128,""longitude"":...",Teitu
27176,Rape estilo Lastres,17.82,Plato,9.1,"""Asturiano""","""C","""geo"":{""latitude"":40.457086513128,""longitude"":...",Teitu


# Data Cleaning

In [12]:
lat = []
log = []
for i in df_rest['geo']:
    lat_long = re.findall(r'[-+]?\d*\.*\d+', i)
    lat.append(lat_long[0])
    log.append(lat_long[1])
df_rest['lat'] = lat
df_rest['log'] = log
df_rest.drop('geo', axis=1, inplace=True)
df_rest.drop('direction', axis=1, inplace=True)

In [13]:
cuisine = []
for i in df_rest['cuisine']:
    cuisine.append(i.replace('"', ''))
df_rest['cuisine'] = cuisine    

In [14]:
#rename columns:
df_rest.columns = ['product', 'price', 'type', 'rating', 'cuisine', 'restaurant', 'lat', 'log']
df_rest

,product,price,type,rating,cuisine,restaurant,lat,log
0,Misto Tagliatella,15.30,Gratinati e Risotto,8.7,Italiano,La Tagliatella - Fuencarral,40.4310901,-3.70347
1,Cannelloni,13.85,Gratinati e Risotto,8.7,Italiano,La Tagliatella - Fuencarral,40.4310901,-3.70347
3,Copa de champán,11.00,Bebidas,9,Americano,La Gamella,40.418802280939,-3.6888999938965
4,Café,2.10,Bebidas,9,Americano,La Gamella,40.418802280939,-3.6888999938965
6,Media botella de agua,3.00,Bebidas,9,Americano,La Gamella,40.418802280939,-3.6888999938965
...,...,...,...,...,...,...,...,...
27169,Wagyu tataki,85.00,Wagyu,9.2,Japonés,Sushi Bar Hannah,40.4298681,-3.6879861
27170,Wagyu hoba,85.00,Wagyu,9.2,Japonés,Sushi Bar Hannah,40.4298681,-3.6879861
27175,Fabes de Tineo con su compango,17.82,Plato,9.1,Asturiano,Teitu,40.457086513128,-3.6921598911285
27176,Rape estilo Lastres,17.82,Plato,9.1,Asturiano,Teitu,40.457086513128,-3.6921598911285


In [15]:
df_rest[df_rest['type']=='Bebidas']

,product,price,type,rating,cuisine,restaurant,lat,log
3,Copa de champán,11.00,Bebidas,9,Americano,La Gamella,40.418802280939,-3.6888999938965
4,Café,2.10,Bebidas,9,Americano,La Gamella,40.418802280939,-3.6888999938965
6,Media botella de agua,3.00,Bebidas,9,Americano,La Gamella,40.418802280939,-3.6888999938965
7,Botella de vino,14.00,Bebidas,9,Americano,La Gamella,40.418802280939,-3.6888999938965
8,Copa de vino,3.75,Bebidas,9,Americano,La Gamella,40.418802280939,-3.6888999938965
...,...,...,...,...,...,...,...,...
27124,Copa de vino,2.80,Bebidas,8.8,Mediterráneo,Gustos Madrid Plaza Mayor,40.4149274,-3.7079592
27131,Café,1.50,Bebidas,None,Chino,Chila,40.4204493,-3.7079859
27132,Botella de agua,2.00,Bebidas,None,Chino,Chila,40.4204493,-3.7079859
27134,Botella de vino,15.00,Bebidas,None,Chino,Chila,40.4204493,-3.7079859


In [16]:
#all values in lowercase:
df_rest = df_rest.apply(lambda x: x.astype(str).str.lower())

In [17]:
df_rest.price.replace(',', '.', inplace=True)

In [18]:
df_rest

,product,price,type,rating,cuisine,restaurant,lat,log
0,misto tagliatella,15.3,gratinati e risotto,8.7,italiano,la tagliatella - fuencarral,40.4310901,-3.70347
1,cannelloni,13.85,gratinati e risotto,8.7,italiano,la tagliatella - fuencarral,40.4310901,-3.70347
3,copa de champán,11.0,bebidas,9,americano,la gamella,40.418802280939,-3.6888999938965
4,café,2.1,bebidas,9,americano,la gamella,40.418802280939,-3.6888999938965
6,media botella de agua,3.0,bebidas,9,americano,la gamella,40.418802280939,-3.6888999938965
...,...,...,...,...,...,...,...,...
27169,wagyu tataki,85.0,wagyu,9.2,japonés,sushi bar hannah,40.4298681,-3.6879861
27170,wagyu hoba,85.0,wagyu,9.2,japonés,sushi bar hannah,40.4298681,-3.6879861
27175,fabes de tineo con su compango,17.82,plato,9.1,asturiano,teitu,40.457086513128,-3.6921598911285
27176,rape estilo lastres,17.82,plato,9.1,asturiano,teitu,40.457086513128,-3.6921598911285


In [19]:
df_rest['product'].value_counts()

copa de vino                                                        691
café                                                                677
botella de vino                                                     652
media botella de agua                                               580
botella de agua                                                     283
                                                                   ... 
ban mian                                                              1
reina margarita                                                       1
enchiladas suizas (2)                                                 1
ramen de setas del bosque, fideo soba y huevo a baja temperatura      1
kokotxas de bacalao con pisto de txangurro                            1
Name: product, Length: 15900, dtype: int64

In [20]:
df_rest['restaurant'].value_counts()

chifa peruano chalán del ...    116
ferreiro                        116
inari - general pardiñas        111
la mafia se sienta a la m...    104
asian                           102
                               ... 
tierra de especias halal          6
el arquibar                       6
timesburg luchana                 6
a vánvera                         5
el tormo                          2
Name: restaurant, Length: 790, dtype: int64

In [21]:
food_type_corrections = {'menú (bebidas excluidas)': 'plato',
                         'tartar, tiraditos, sashimis': 'postre',
                         'platos elaborados': 'plato',
                         'bebidas': 'bebida', 
                         'entrantes': 'entrante', 
                         'postres': 'postre', 
                         'nuestras salsas caseras': 'salsa',
                         'ensaladas': 'ensalada', 
                         'vegetarianos': 'plato - vegetariano', 
                         'recomendaciones': 'plato', 
                         'especialidades queen': 'plato', 
                         'arroces y fideua': 'plato - arroz', 
                         'gratinati e risotto': 'plato - risotto', 
                         'huevos con fundamento, con la yema bien amarillita': 'plato',
                         'salteado de arroz al wok con verduras de temporada': 'plato - arroz',
                         'arroz y tallarines': 'plato - arroz',
                         'ensaladas y sopas': 'ensalada',
                         'tapas y raciones': 'plato', 'carnes y pescados': 'plato',
                         'ensaladas y verduras': 'ensalada',
                         'pescados y mariscos': 'plato', 
                         'verduras y tofu': 'plato',
                         'hot-dog y costillas - acompañados con patatas fritas o nachos orlando': 'plato',
                         'guiso y nabe': 'plato', 
                         'zumos y batidos': 'bebida', 
                         'tortilla de espinaca y patatas confitadas': 'plato',
                         'cordero curry': 'plato', 'arroces y pescados': 'plato', 
                         'arroces y fideo': 'plato - arroz', 
                         'zumos naturales y smoothies': 'bebida', 
                         'entrantes y primeros platos': 'plato', 
                         'platos y sancocho': 'plato', 
                         'parrillas y braseria': 'plato', 
                         'zumos y cocteles de frutas': 'bebidas', 
                         'arroces y pastas': 'plato', 
                         'risotti / arroces ( min 2 personas)': 'plato - arroz', 
                         'platos de biryani': 'plato', 
                         'platos principales -pollo a la brasa': 'plato',
                         'curry con vegetales y quinoa roja bio salteada': 'plato',
                        'entremeses y sopas': 'plato',
                        'chacinas finas y quesos gallegos': 'entrante',
                        'pasta tadizionale - elige la variedad de pasta que más te gusta y acompáñala con una de nuestras salsas artesanales': 'plato',
                        'pregunte por nuestros guisos y platos del día': 'plato',
                        'pollo de corral y pato': 'plato',
                        'verduras curry': 'plato',
                        'con los mejores huevos camperos': 'plato',
                        'menu infantil': 'plato',
                        'arroces y pasta': 'plato',
                        'comidas y cenas': 'plato',
                        'sushi bar nigiris y sashimi': 'plato',
                        'mariscos y pescados': 'plato',
                        'menú sushi y sashimi': 'plato',
                        'nuestras ensaladas y guarniciones': 'ensalada',
                        'o crea tu propia hamburguesa - todas las hamburguesas se sirve con base de lechuga, tomate y cebolla morada': 'plato',
                        'verduras y ensaladas': 'ensalada',
                        'calzone y stromboli': 'salsa',
                        'pizzas y calzones': 'pizza',
                        'platos de tandoori (especialidades)': 'plato',
                        'menú del dia': 'plato',
                        'nuestros txuletones': 'plato',
                        'guarnicion':'plato',
                        'taiyaki': 'postre',
                        'cordero (curries)': 'plato',
                        'postres sin gluten': 'postre',
                        'gambas curry': 'plato',
                        'menú degustación':' plato',
                        'los guisos': 'plato',
                        'de la huerta a la mesa': 'plato',
                        '¿pedimos una ensalada?': 'ensalada',
                        'gyoza artesanas de rabo de toro con foie': 'plato',
                        'individuales para empezar': 'entrante',
                        'tartare - tartare': 'plato',
                        'pollo (curries)': 'plato',
                        'los pulpos de mila': 'plato',
                        'maki (6 unidades)': 'plato',
                        'paste fresche e secche / pastas frescas y secas': 'plato',
                        'embutidos, compartir y quesos': 'entrante',
                        'de nuestros campos y mares': 'plato',
                        'raciones y platos': 'plato',
                        'recetas vegetarianos y veganos': 'plato',
                        'tostas (con pan cristal) y sandwiches': 'plato',
                        'carnes y aves': 'plato',
                        'empezar y compartir / to start and share': 'entrante',
                        'entrantes, ensaladas y verduras': 'entrante - ensalada',
                        'entradas y picoteo': 'entrante',
                        'carnes y pesacados': 'plato',
                        'sandwich y bagel (all day)': 'plato',
                        'selezione di carne': 'plato',
                        'nuestra parrilla de carbón': 'plato',
                        'productos del mar': 'plato',
                        'tapiocas (tortilla de harina de yuca. sin gluten)': 'plato',
                        'verduras y legumbres': 'plato',
                        'tallarines y arooz':'plato',
                        'nuestros arroces y fideuás':'plato',
                        'pasta ripiena - elige la variedad de pasta que más te gusta y acompáñala con una de nuestras salsas artesanales':'plato',
                        'arroz y noodles': 'plato - arroz',
                        'tartares y sashimis': 'plato',
                        'desayuno y merienda': 'entrante',
                        'sopas y guisos': 'plato',
                        'primeros y segundos': 'plato',
                        'sopas y arroces': 'plato',
                         'hamburguesas y hot dogs': 'plato',
                        'aves y carnes':' plato',
                        'sugerecias en ensaladas': 'ensalada',
                        'pastas y arroces': 'plato',
                        'aves y caza': 'plato',
                        'entrantes frios y ensaladas': 'entrante - ensalada',
                        'pescado y marisco': 'plato',
                        'sugerencias de tapas y raciones': 'plato',
                        'arroces y noodles': 'plato',
                        'de los valles y montañas de asturias': 'plato',
                        'tou fu': 'plato',
                        'pato lakeado crispy': 'plato',
                        'platos caseros': 'plato',
                        'tortillitas (all day)': 'plato',
                        'nuestros sorbetes': 'plato',
                        'ensaladas verduras y hortalizas': 'ensalada',
                        'ensaladas, pasta y arroces': 'ensalada - plato',
                        'platos tradicionales persa':'plato',
                        'miss éxitos en rolls tempurizados':'plato',
                        'plato do de tandoori / horno': 'plato',
                        'sánwiches': 'plato',
                        'nuestros guisos del día...': 'plato',
                        'croquetas…..': 'plato',
                        'de la montaña y las granjas': 'plato',
                        'dulces y goloseo….':'postre',
                        '…zuria y el mar': 'plato',
                        'perros y wraps':'plato',
                        'rollitos dumpling y bahn cuon': 'plato',
                        'pollo curry': 'plato',
                        'tartar, tiraditos y sashimi': 'plato',
                        'verdes y frescos': 'plato',
                        'hamburguesas y perritos': 'plato',
                        'cremas, ensaladas y otros platos frios': 'ensalada - plato',
                        'sopa y ramen': 'plato',
                        'bocadillos y sandwiches': 'plato',
                        'platos de gambas y pescado': 'plato',
                        'langostinos curry': 'plato',
                        'guarnición': 'plato',
                        'diseña tu bowl con tus ingredientes favoritos': 'plato',
                        'caldosos (mínimo dos personas, precio por persona)': 'plato',
                        'la empanada y las tortillas': 'plato',
                        'blinis y nalisniki': 'postre',
                        'chhurpi; marinado con queso nata,especias y hierbas.':'plato',
                        'safari; marinado con yogur, menta, especias y hierbas.': 'plato',
                        'para los peques': 'plato',
                        'huerta':'plato',
                        'los postres': 'postre',
                        'cazuela caliente':'plato',
                        'nigiri sushi':'plato',
                        'entrantes y ensalada': 'ensalada - entrante',
                        'arroz y fideos': 'plato - arroz',
                        'hamburguesas (escoge tu carne y van acompañadas con patatas graal)':'plato',
                        'la cuchara y el puchero': 'plato',
                        'arroz, pescados y mariscos': 'plato',
                        '…carnes y guisos': 'plato',
                        'dulces divertidos y de colores':'postre',
                        'beef':'plato',
                        'menús de temporada 2018':'plato',
                        'pastas y fideos':'plato',
                        'platos combinados y parrilla':'plato',
                        'pepitos ((escoge tu carne y van acompañadas con patatas graal))':'plato',
                        'ensaladas y cremas':'ensalada - crema',
                        'ensaladas y entrantes':'ensalada - entrante',
                        'kathmandu dishes; preparado con nueces, nata, carne picada y huevos con sabor especial.':'plato',
                        'variantes y ultramarinos': 'plato',
                        'chacinas y salazones': 'entrante',
                        'embutidos y salazones': 'entrante',
                        'chacinas, quesos y salazones':'entrante',
                        'chacinas, salazones y foie': 'entrante',
                        'enchiladas':'plato',
                        'ternera (curries)':'plato',
                        'wraps':'plato',
                        'embutidos y quesos':'entrante',
                        'ensaladas y verduras a la parrilla': 'ensalada',
                        'postres y bebidas': 'postre - bebida',
                        'arroces y entrantes': 'entrante',
                        'ensaladas frescas y verduras': 'ensalada',
                        'raciones y sugerencias para compartir ... o no': 'plato',
                        'pasta y arroces': 'plato',
                        'pahadi; marinado con chili, ajo,jengibre, especias, hierbas y asado en el horno tandoori': 'plato',
                        'arroces (mínimo 2 p.) precio por persona elaborados con el mejor arroz de calasparra. arroces secos y caldosos':'plato - arroz',
                        'platos para acompañar y aperitivos':'plato',
                        'emtrantes y fritos':'entrante',
                        'patatas y huevos':'entrante',
                        'pescados, carnes y más':'plato',
                        'cremas y sopas':'plato',
                        'pescados y carnes':'plato',
                        'carne y pescado':'plato',
                        'tostas y entrepanes':'plato',
                        'sopas y caldos':'plato',
                        'carnes y guisos…..':'plato',
                        'zuría y el mar…..':'plato',
                        'pasta y arroz':'plato',
                        'fondues y carnes':'plato',
                        'mariscos y pescados salvajes':'plato',
                        'verduras y hortalizas':'plato',
                        'pizzas y hamburguesas':'plato',
                        'pahadi; marinado con chili, ajo,jengibre, especias, hierbas y asado en el horno tandoori.':'plato',
                        'arroces y paellas':'plato',
                        'noodles y arroces':'plato - arroz',
                        'arroz y pasta':'plato',
                        'tallarines y arroces': 'plato - arroz',
                        'verdes, frescas y curiosas':'plato',
                        'lumbini sekuwa; cocinando con pimientos, cebolla, tomate, ajo jengibre y hierbas.': 'plato',
                        'tandoori especial (plato asado en plancha caliente, marinado con yogur, jengibre, ajos, limón y especias)': 'plato',
                        'tortas y burritos':'plato',
                        'ensaladas frescas y verduras....':'ensalada',
                        'huevería y croquetas':'plato',
                        'nuestra especialidad: el arroz y la paella':'plato - arroz',
                        'caviar y ahumados':'plato',
                         'sugerencias del día y platos de temporada':'plato',
                        'condimentos y salsas':'salsa',
                        'himalaya ross dishes; preparado con frutos secos, salsa de tomate, nata y hierbas de himalaya.':'plato',
                        'everest chilli garlic dishes; preparado con chili,ajo, pimientos,cebolla, tomate, cilantro, especias y hierbas.':'plato',
                        'achari; marinado al estilo tikka, con salsa incurtdos ácidos y asado en el horno tandoori.':'plato',
                        'quesos e ibéricos del país (con picos andaluces y pan recién tostado)':'entrante',
                        'mustang dishes; preparado con salsa de anacardos, hierbas, especias, nata y loquera de manzana.':'plato',
                        'mango dishes; cocinando con mango, nata almendras,especias y hierbas.': 'plato',
                        'nuestros arroces y fideuá (mín. 2 personas) precio por persona - hacemos los arroces secos y caldosos':'plato',
                        'verduras, primeros platos y arroces': 'plato',
                        'mint sekuwa; cocinando con yogur, menta fresca, ajo jengibre y hierbas de himalaya.':'plato',
                        'pokhara garlic; rebozado y cocinado con ajo jengibre,yogur, pimiento,cebolla, especias y hierbas.':'plato',
                        'kesar dishes; preparado con almendras, tomate, salsa curry, nata, hierbas y especias.':'plato',
                        'castizo y de cuchara.':'plato',
                        'kasmiri dishes; preparado con fruta mixta, almendras, yogur, nata, especias y hierbas':'plato',
                        'kasmiri dishes; preparado con fruta mixta, almendras, yogur, nata, especias y hierbas.':'plato',
                        'sopas y cremas': 'plato',
                        'chacinas finas, quesos y salazones':'entrante',
                        'green valley dishes; preparado con hierbas de himalaya, espinacas nueces y menta.':'plato',
                        'sushis combinados':'plato',
                        'guiso tradicional japonés':'plato',
                        'japanese wagyu beef':'plato',
                        'ceviches y tiraditos':'plato',
                        'arroz y fideo':'plato',
                        'fondues y raclettes':'plato',
                        'tostas (all day)':'plato',
                         'bandejas':'entrante',
                        'nuestros bowls':'plato',
                        'frescas ensaladas':'ensalada',
                        'mare e monti':'plato',
                        'dulces':'postre',
                        'irresistibles postres':'postre',
                        'dolci artigianali':'postre',
                        'sushi & sashimi variado':'plato',
                        'uramaki':'plato',
                        'platos veggie':'plato',
                        'nuetras hamburguesas':'plato',
                        'milanesas':'plato',
                        'dumplings':'plato',
                        'nuestros helados':'postre'}

In [22]:
for k, v in food_type_corrections.items():
    df_rest.type.replace(k, v, inplace=True)

In [23]:
df_rest[df_rest['type'].str.contains(' y ', regex=False)]['type'].value_counts()

entrantes y raciones                                                          18
nuestras entradas y raciones                                                  17
pizzas crujientes y de masa fina                                              16
entradas y raciones                                                           15
le pizze ** solo se sirven pizzas en horario de cenas y en fines de semana    14
                                                                              ..
huevos pa,romper y patatas                                                     1
sushi y sashimi                                                                1
falso risotto de espinacas con leche de coco y lascas de parmesano             1
huevos, patatas y más…                                                         1
brotes verdes, remolacha, pollo de campo, kale y almendras.                    1
Name: type, Length: 64, dtype: int64

In [24]:
for plate in df_rest['type']:
    if 'plato' in plate or 'platos' in plate or 'parrilla' in plate or 'temporada' in plate or 'clásicos' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'ensalada' in plate or 'insalate' in plate:
        df_rest.type.replace(plate, 'ensalada', inplace=True)
    elif 'carne' in plate or 'carnes' in plate or 'steak' in plate or 'mar' in plate or 'calientes' in plate or 'empanada' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'pescado' in plate or 'burgers' in plate or 'burger' in plate or 'cocina' in plate or 'muy crudo' in plate or 'perú' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'arroces' in plate or 'arroz' in plate or 'raciones' in plate or 'cooking' in plate or 'tosta' in plate or 'basmati' in plate or 'burguer' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'crema' in plate or 'sopa' in plate or 'princpales' in plate or 'primeros' in plate or 'receta' in plate or 'comida' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'dulce' in plate or 'dulces' in plate or 'postre' in plate:
        df_rest.type.replace(plate, 'postre', inplace=True)
    elif 'pasta' in plate or 'pizza' in plate or 'tortilla' in plate or 'principales' in plate or 'sashimi' in plate or 'hamburguesa' in plate or 'al vapor' in plate or 'tallarin' in plate or 'maki' in plate or 'dim sum' in plate or 'costilla' in plate or 'ceviche' in plate or 'cuchara' in plate or 'tandoori' in plate or 'cerdo' in plate or 'brasa' in plate or 'ave' in plate or 'wok' in plate or 'chicken' in plate or 'forno' in plate or 'risotti' in plate or 'verde' in plate or 'plancha' in plate or 'horno' in plate or 'ramen' in plate or 'lobster' in plate or 'grill' in plate or 'cook' in plate or 'huerta' in plate or 'secondi piatti' in plate or 'primi piatti' in plate or 'chef' in plate or 'asturia' in plate or 'salmón' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'sushi' in plate or 'sandwiches' in plate or 'sándwiches' in plate or 'hmburguesas' in plate or 'segundos' in plate or 'huevos' in plate or 'tempura' in plate or 'arepas' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'marisco' in plate or 'verduras' in plate or 'verdura' in plate or 'paella' in plate or 'acompañamientos' in plate or 'continua' in plate or 'tierra' in plate or 'carnicería' in plate or 'frito' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'bollería' in plate or 'helado' in plate or 'tarta' in plate or 'crep' in plate or 'dolci' in plate:
        df_rest.type.replace(plate, 'postre', inplace=True)
    elif 'cordero' in plate or 'rolls' in plate or 'wraps' in plate or 'fries' in plate or 'pig' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'de la huerta' in plate or 'pollo' in plate or 'especiales' in plate or 'bocadillos' in plate or 'especialidade' in plate or'racione' in plate or 'comer' in plate or 'curry' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'para picar' in plate or 'compartir' in plate or 'tapea' in plate or 'entrantes' in plate or 'aperitivos' in plate or 'tapas' in plate or 'comenzar' in plate or 'empezamos' in plate or 'empezar' in plate or 'entrada' in plate or 'antipasti' in plate or 'pica' in plate or 'hummus' in plate or 'pikoteo' in plate or 'empezando' in plate or 'antipasto' in plate or 'tapeo' in plate or 'picoteo' in plate or 'aperritivos' in plate or 'barra' in plate or 'tapita' in plate or 'tapeando' in plate or 'para abrir boca' in plate:
        df_rest.type.replace(plate, 'entrante', inplace=True)
    elif 'pescadería' in plate or 'solo para mí' in plate or 'bacalao' in plate or 'guarniciones' in plate or 'ternera' in plate or 'hmburguesas' in plate or 'pato' in plate or 'nigiri' in plate or 'roll' in plate or 'croqueta' in plate or 'sandwich' in plate:
        df_rest.type.replace(plate, 'plato', inplace=True)
    elif 'vino' in plate or 'cerveza' in plate or 'coctelería' in plate or 'café' in plate or 'cafe' in plate or 'infusiones' in plate or 'bebida' in plate or 'cocktail' in plate or 'rioja' in plate or 'smoothies' in plate or 'jugos' in plate or 'cócteles' in plate:
        df_rest.type.replace(plate, 'bebida', inplace=True)

In [25]:
df_rest['type'].value_counts()

plato                                                                   9983
entrante                                                                4341
bebida                                                                  3523
postre                                                                  2834
ensalada                                                                 814
                                                                        ... 
udon and soba                                                              1
batidos naturales ( precio no aplicable a promociones ni descuentos)       1
de las gallinas                                                            1
suplemento de espectáculo                                                  1
menú mediodia de 13 a 16                                                   1
Name: type, Length: 428, dtype: int64

In [26]:
df_rest = df_rest[df_rest.groupby('type').type.transform(len) > 570]

In [27]:
for i in df_rest[df_rest['type']=='bebida']['product']:
    print(i)

copa de champán
café
media botella de agua
botella de vino
copa de vino
café
botella de agua
media botella de agua
botella de champán
copa de champán
café
botella de agua
media botella de agua
botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
botella de agua
café
botella de agua
media botella de agua
botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
botella de champán
café
media botella de agua
botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
café
botella de agua
botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
café
media botella de agua
copa de vino
copa de champán
media botella de agua
botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
café
botella de agua
botella de vino
copa de vino
café
botella de agua
media botella de agua
botella de vino
copa de vino


botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
botella de champán
copa de champán
café
media botella de agua
botella de vino
copa de vino
botella de champán
café
botella de agua
media botella de agua
botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
café
media botella de agua
botella de vino
copa de vino
botella de champán
copa de champán
botella de vino
copa de vino
café
botella de agua
botella de vino
copa de vino
1 copa rivera del duero crianza
1 copa rioja crianza
1 copa berdejo
1 copa rueda
1 botella rioja o ribera del duero crianza
1 botella berdejo o rueda
tinto verano
jarra de sangria


In [28]:
good_dict = {'á': 'a', 'à': 'a', 'é':'e', 'í':'i', 'ó': 'o', 'ú': 'u'}
df_rest.replace(good_dict, regex=True, inplace=True)

/home/almsasantos/miniconda3/envs/final_project/lib/python3.8/site-packages/pandas/core/frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [29]:
df_rest.to_csv('restaurant_dataframe.csv')

In [30]:
df_rest

,product,price,type,rating,cuisine,restaurant,lat,log
0,misto tagliatella,15.3,plato,8.7,italiano,la tagliatella - fuencarral,40.4310901,-3.70347
1,cannelloni,13.85,plato,8.7,italiano,la tagliatella - fuencarral,40.4310901,-3.70347
3,copa de champan,11.0,bebida,9,americano,la gamella,40.418802280939,-3.6888999938965
4,cafe,2.1,bebida,9,americano,la gamella,40.418802280939,-3.6888999938965
6,media botella de agua,3.0,bebida,9,americano,la gamella,40.418802280939,-3.6888999938965
...,...,...,...,...,...,...,...,...
27164,yakibifun apto para celiacos,11.25,plato,8.7,japones,janatomo,40.4200447,-3.6989747
27165,pollo crocante,9.0,ensalada,8.7,mediterraneo,monss,40.5201552,-3.7778506
27175,fabes de tineo con su compango,17.82,plato,9.1,asturiano,teitu,40.457086513128,-3.6921598911285
27176,rape estilo lastres,17.82,plato,9.1,asturiano,teitu,40.457086513128,-3.6921598911285
